# KM3NeT Masterclass Analysis

Upload the calibration csv file, then click **Run analysis** to execute `analysis.py`.


In [ ]:
%matplotlib inline

from pathlib import Path

import ipywidgets as widgets
from IPython.display import display, clear_output

from analysis import run_analysis

# ---- Paths configuration ----
# This should be the input sky-map CSV already in your repo,
# with columns 'ra_rad' and 'dec_rad'.
# CHANGE THIS to the correct file name / path if needed.
INPUT_CSV = "sky_map.csv"   # e.g. "data/sky_map.csv"

# Output CSV name (the script will also create PNGs based on this)
OUTPUT_CSV = "scrambled_events.csv"

# Where we will save the uploaded calibration file
CALIBRATION_DEST = Path("config") / "calibration_nearly_perfect.csv"

In [ ]:
# ---- Widgets ----

upload = widgets.FileUpload(
    accept=".csv",     # only CSV files
    multiple=False      # single file
)

run_button = widgets.Button(
    description="Run analysis",
    button_style="primary",  # 'success', 'info', 'warning', 'danger' or ''
    tooltip="Run analysis.py with the uploaded calibration file"
)

output_area = widgets.Output()


def on_run_clicked(btn):
    with output_area:
        clear_output()
        
        # Check that the user uploaded a file
        if not upload.value:
            print("⚠ Please upload a calibration CSV file first.")
            return
        
        # Handle both ipywidgets 7 (dict) and 8+ (tuple) formats
        val = upload.value

        if isinstance(val, dict):
            # Old style: dict of {filename: {'metadata': ..., 'content': ...}}
            file_info = next(iter(val.values()))
            filename = file_info["metadata"]["name"]
            content = file_info["content"]
        else:
            # New style: tuple of UploadedFile objects
            file_info = val[0]
            filename = file_info.name
            content = file_info.content
        
        print(f"Received file from user: {filename}")
        
        # Make sure the config/ folder exists
        CALIBRATION_DEST.parent.mkdir(parents=True, exist_ok=True)
        
        # Save uploaded file to the canonical path the code expects
        CALIBRATION_DEST.write_bytes(content)
        print(f"Saved uploaded calibration to: {CALIBRATION_DEST}")
        
        # Run the main analysis
        print("\nRunning analysis.py ...\n")
        try:
            result_outfile = run_analysis(
                infile=INPUT_CSV,
                outfile=OUTPUT_CSV,
                calibration_file=str(CALIBRATION_DEST)
            )
        except Exception as e:
            print("❌ Error while running analysis:")
            print(e)
            return
        
        print("\n✅ Analysis finished.")
        print(f"Output CSV: {result_outfile}")
        print("Plots should appear above or below this cell (depending on your environment).")


# Wire the button to the callback
run_button.on_click(on_run_clicked)

# Display the UI
ui = widgets.VBox([
    widgets.HTML("<h2>Upload calibration file and run analysis</h2>"),
    upload,
    run_button,
    output_area
])

display(ui)